In [ ]:
# Imports
import cv2
import os
import numpy as np
from cvzone.HandTrackingModule import HandDetector
from skimage.morphology import skeletonize
from skimage.measure import label, regionprops

# Projet - Air Drawing
---
EXPLICATION DE LA PROBLEMATIQUE

## Datsets

### EMNIST
EXPLICATION

### AUTRE DATASET
EXPLICATION

## Entrainement de lettres

### Option A (CNN Maison)

#### CCN Maison 1

##### Modèle

In [ ]:
# TO DO

##### Tableau d'apprentissage

In [ ]:
# TO DO

#### CCN Maison 2

##### Modèle

In [ ]:
# TO DO

##### Tableau d'apprentissage

In [ ]:
# TO DO

### Option B (Transfert Learning)

#### Transfer Learning 1 (XXX)

##### Modèle Feature Extraction

In [ ]:
# TO DO

##### Tableau d'apprentissage (Feature Extraction)

In [ ]:
# TO DO

##### Modèle Fine-tuning

In [ ]:
# TO DO

##### Tableau d'apprentissage (Fine-tuning)

In [ ]:
# TO DO

#### Transfer Learning 2 (XXX)

##### Modèle Feature Extraction

In [ ]:
# TO DO

##### Tableau d'apprentissage (Feature Extraction)

In [ ]:
# TO DO

##### Modèle Fine-tuning

In [ ]:
# TO DO

##### Tableau d'apprentissage (Fine-tuning)

In [ ]:
# TO DO

## Évaluation Comparative et Analyse Critique
Explication et résumer des résultat avec les tableaux etc
Meilleur modele dans quel cas et pourquoi (temps, MSE, accuracy, etc)

## Vidéo

In [ ]:
# === Fonctions utilitaires ===
def vider_dossier(dossier):
    if os.path.exists(dossier):
        for f in os.listdir(dossier):
            chemin = os.path.join(dossier, f)
            if os.path.isfile(chemin):
                os.remove(chemin)
    else:
        os.makedirs(dossier)

# === 0. Définition des chemins ===
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

print(BASE_DIR)
video_path = os.path.join(BASE_DIR, 'videos/Lettres/I.mp4')
extracted_dir = os.path.join(BASE_DIR, 'images_extraites')
finger_dir = os.path.join(BASE_DIR, 'finger_find')
frame_interval = 2

# === 1. Nettoyage des dossiers ===
vider_dossier(extracted_dir)
vider_dossier(finger_dir)

# === 2. Extraction des frames ===
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Erreur : impossible d'ouvrir la vidéo '{video_path}'")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Vidéo chargée : {total_frames} frames à {fps:.2f} fps")

frame_count = 0
saved_count = 0

while True:
    success, frame = cap.read()
    if not success:
        break

    if frame_count % frame_interval == 0:
        filename = os.path.join(extracted_dir, f"frame_{saved_count:04d}.jpg")
        cv2.imwrite(filename, frame)
        saved_count += 1

    frame_count += 1

cap.release()
print(f"{saved_count} images extraites dans le dossier '{extracted_dir}'")

# === 3. Détection du bout de l'index ===
trace_points = []
img_shape = None
detector = HandDetector(staticMode=True, maxHands=1, detectionCon=0.7)

#for filename in os.listdir(extracted_dir):
for filename in sorted(os.listdir(extracted_dir)):
    if not filename.endswith(('.jpg', '.png')):
        continue

    image_path = os.path.join(extracted_dir, filename)
    image = cv2.imread(image_path)
    hands, img = detector.findHands(image)

    if hands:
        hand = hands[0]
        lm_list = hand['lmList']
        if len(lm_list) >= 9:
            x, y = lm_list[8][0], lm_list[8][1]
            trace_points.append((x, y))

    #cv2.imwrite(os.path.join(finger_dir, filename), img)

# === 4. Génération de l'image composite ===
#sample_img = cv2.imread(os.path.join(finger_dir, os.listdir(finger_dir)[0]))
if img_shape is None:
    img_shape = image.shape

height, width, _ = img_shape
result = np.zeros((height, width, 3), dtype=np.uint8)

for i in range(1, len(trace_points)):
    cv2.line(result, trace_points[i - 1], trace_points[i], (0, 0, 255), thickness=6)

#for filename in os.listdir(finger_dir):
#    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#        path = os.path.join(finger_dir, filename)
#        img = cv2.imread(path)
#        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#
#        lower_red1 = np.array([0, 100, 100])
#        upper_red1 = np.array([10, 255, 255])
#        lower_red2 = np.array([160, 100, 100])
#        upper_red2 = np.array([180, 255, 255])
#
#        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
#        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
#        mask = mask1 | mask2
#
#        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#        if contours:
#            biggest = max(contours, key=cv2.contourArea)
#            if cv2.contourArea(biggest) > 0:
#                cv2.drawContours(result, [biggest], -1, (0, 0, 255), -1)

# === 5. Rotation de 90° vers la droite ===
rotated = cv2.rotate(result, cv2.ROTATE_90_CLOCKWISE)

# === 6. Effet miroir (symétrie horizontale) ===
mirrored = cv2.flip(rotated, 1)

# === 7. Sauvegarde de l'image finale ===
cv2.imwrite("../image_resultat.png", mirrored)
print("Image finale enregistrée sous 'image_resultat.png' (rotation + effet miroir)")


## Traitements d'image

In [ ]:
def save_step_logs(image, name, output_dir="debug_steps"):
    os.makedirs(output_dir, exist_ok=True)
    if image is None or image.size == 0:
        print(f"[WARNING] Cannot save '{name}': image is empty.")
        return
    cv2.imwrite(os.path.join(output_dir, f"{name}.png"), image)

# Step 1: Extract red from image
def extract_red_mask(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    return cv2.bitwise_or(mask1, mask2)

# Step 2: Basic cleaning (open/close)
def clean_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# Step 3: Keep relevant parts
def filter_components(mask, min_area=50):
    labeled = label(mask)
    cleaned = np.zeros_like(mask)
    for region in regionprops(labeled):
        if region.area >= min_area:
            for y, x in region.coords:
                cleaned[y, x] = 255
    return cleaned

# Step 4: Skeletonize
def get_skeleton(mask):
    return (skeletonize(mask > 0) * 255).astype(np.uint8)

# Step 5 : Bold the ligne
def thicken_mask(mask, size=3):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    return cv2.dilate(mask, kernel, iterations=1)

# Step 6: Resize and center
def center_and_resize(mask, output_size=28, margin=2):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return np.ones((output_size, output_size), dtype=np.uint8) * 255
    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    cropped = mask[y:y+h, x:x+w]
    resized = cv2.resize(cropped, (output_size - 2 * margin, output_size - 2 * margin))
    canvas = np.ones((output_size, output_size), dtype=np.uint8) * 255
    cx = (output_size - resized.shape[1]) // 2
    cy = (output_size - resized.shape[0]) // 2
    canvas[cy:cy + resized.shape[0], cx:cx + resized.shape[1]] = 255 - resized

    return canvas

# Main function
def process_image(path, output_dir="debug_steps"):
    img = cv2.imread(path)
    save_step_logs(img, "00_original", output_dir)

    mask = extract_red_mask(img)
    save_step_logs(mask, "01_red_mask", output_dir)

    cleaned = clean_mask(mask)
    save_step_logs(cleaned, "02_cleaned", output_dir)

    filtered = filter_components(cleaned)
    save_step_logs(filtered, "03_filtered", output_dir)

    skeleton = get_skeleton(filtered)
    save_step_logs(skeleton, "04_skeleton", output_dir)

    thickened = thicken_mask(skeleton, size=50)
    save_step_logs(thickened, "05_thickened", output_dir)

    final = center_and_resize(thickened)
    save_step_logs(final, "06_final", output_dir)

    print("[INFO] Simplified processing complete.")
    cv2.imwrite(os.path.join("../Resultats/Conversion", "result.png"), final)

    return final

# Run on your image
process_image("../image_resultat.png")

## Test "Réel"

In [ ]:
# TO DO

## Conclusions et Décisions